In [ ]:
%load_ext autoreload
%autoreload 2
#%pip install torchvision
import random
from pathlib import Path
from matplotlib import pyplot as plt

from cvla.data_loader_h5 import H5Dataset
from cvla.data_loader_jsonl import JSONLDataset
from cvla.data_augmentations import augment_image_rgb, RandomizeBackgrounds
import torchvision.transforms



dataset_location = Path("/tmp/clevr-act-7-depth")
real_image_size = 720, 1280
resize_cropper = torchvision.transforms.RandomResizedCrop(size=real_image_size, scale=(0.9, 1.0),)
jsonl_file_path = Path("/data/lmbraid19/argusm/datasets/cvla-droid-1of5c-v1/_annotations.train.jsonl")
bg_img_dataset = JSONLDataset(jsonl_file_path=jsonl_file_path, augment_rgb=resize_cropper)
train_dataset = bg_img_dataset
randomize_background = RandomizeBackgrounds(p=0.2, background_images = bg_img_dataset)

#randomize_background = RandomizeBackgrounds(p=0.2, background_images_path = "/data/lmbraid19/argusm/datasets/indoorCVPR/Images")
train_dataset = H5Dataset(dataset_location, augment_rgbds=randomize_background, augment_rgb=augment_image_rgb)
print("load done.")

r,c = 2,4
fig, axes = plt.subplots(r, c, figsize=(c*4, r*4))
for i in range(r):
    for j in range(c):
        random_idx = random.choice(range(len(train_dataset)))
        image = train_dataset[random_idx][0]
        ax = axes[i][j]
        ax.axis("off")
        ax.imshow(image)
plt.tight_layout()
plt.show()

# Clean Real Dataset

In [ ]:
%load_ext autoreload
%autoreload 2
dataset_location = "/data/lmbraid19/argusm/datasets/cvla-droid-block-v1"

from tqdm.notebook import tqdm
from cvla.utils_vis import render_example
from cvla.utils_trajectory import DummyCamera
from cvla.data_loader_jsonl import JSONLDataset

test_dataset = JSONLDataset(
    jsonl_file_path=f"{dataset_location}/_annotations.valid.jsonl",
    image_directory_path=f"{dataset_location}/dataset",
)

test_samples = 10#len(test_dataset)
decode_dataset = [None, ]*test_samples
pred_list = []
html_imgs = ""

for i in tqdm(range(test_samples), total=test_samples):
    image, sample = test_dataset[i]

    html_img = render_example(image, label=sample["suffix"], text=sample["prefix"], camera=sample["camera"])
    html_imgs += html_img

plot_images = True
if plot_images:
    from IPython.display import display, HTML
    display(HTML(html_imgs))